Doing Import for Sqlite and modules for webscraping

In [1]:
import sqlite3
import requests

from bs4 import BeautifulSoup 
from bs4.element import Comment



import datetime  
import re 
from collections import Counter, defaultdict
import nltk

In [2]:
sites=[] #writing all the sites to a list
with open("repo_sites.txt",'r')as ifile:
    for line in ifile:
        sites.append(line.strip())

In [3]:
sites

['http://sustainability.stryker.com/',
 'https://www.jnjmedicaldevices.com/en-US/service/reprocessing']

In [4]:
db = sqlite3.connect("reprocessing_competitors.db") # creating connection to DB
cur = db.cursor()

In [5]:
# Creating table in reporcessing_competitors.db
cur.execute('''DROP TABLE IF EXISTS site_text''')
cur.execute('''CREATE TABLE site_text (
    dt DATETIME, 
    base_url TEXT, 
    url TEXT,
    text TEXT,
    pulled BOOLEAN)''')
db.commit()

In [6]:
# pulling all tags, putting them in a list and uploading
for repo_sites in sites:
    r = requests.get(repo_sites)
    soup = BeautifulSoup(r.text, 'html.parser')
    all_a_tags = soup.find_all('a')
    
    links = [repo_sites] 
    
    for link in soup.find_all('a'):
        links.append(link.get('href'))
        
    for link in set(links) : #set gets rid of duplicate links
        new_row = [datetime.datetime.now(),
                   repo_sites,
                   link,
                   "", # empty string for text
                   False]
        
        cur.execute('''INSERT INTO site_text (dt,base_url,url,text,pulled) 
               VALUES (?,?,?,?,?)''',new_row) # inserting list into our DB
        
db.commit()
    

In [7]:
# checking how many links of each website we pulled
results = cur.execute('''SELECT base_url, count(*) as cnt FROM site_text GROUP BY base_url ORDER BY cnt DESC''')

for row in results :
    print("Original site {} has {} links within it.".format(row[0],row[1]))

Original site https://www.jnjmedicaldevices.com/en-US/service/reprocessing has 97 links within it.
Original site http://sustainability.stryker.com/ has 62 links within it.


In [8]:
# function from class that gets rid of all the unledgibale text types like headers and footers
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [9]:
links


['https://www.jnjmedicaldevices.com/en-US/service/reprocessing',
 '#main-content',
 '/en-US',
 '/en-US/home',
 '/en-US',
 '#',
 'https://www.jnjmedicaldevices.com/en-US/companies/acclarent',
 'https://www.biosensewebster.com',
 'http://www.cerenovus.com/',
 'https://www.jnjmedicaldevices.com/en-US/companies/depuy-synthes',
 '/en-US/companies/ethicon',
 '/en-US/companies/mentor',
 'https://www.jnjmedicaldevices.com/en-US/service/reprocessing',
 '/en-US/products',
 '#',
 '/en-US/products',
 '/en-US/product-family/access',
 '/en-US/product-family/adjunctive-hemostats',
 '/en-US/product-family/airway-balloon-dilation-system',
 '/en-US/product-family/balloon-sinuplasty',
 '/en-US/product-family/breast-augmentation',
 '/en-US/product-family/breast-reconstruction',
 'https://www.jnjmedicaldevices.com/en-US/products?jnjmdc_product_franchise_target_id%5B76%5D=76&jnjmdc_product_category_ref_target_id%5B2191%5D=2191&items_per_page=20',
 '/en-US/product-family/energy-sealing-dissecting',
 '/en-US/

In [10]:
max_pages = -1 #putting a limit on the pages

do_break = False

for repo_sites in sites :
    
    
    links = []
    
    sql_query = "SELECT url FROM site_text WHERE base_url = '{}' AND pulled = 0".format(repo_sites)    
    link_results = cur.execute(sql_query)

    for row in link_results :
        links.append(row[0])
    
            
    #If max_pages is non-negative take a sample of the links
    if max_pages < 1 or len(links) < max_pages :
        pages_to_pull = links
    else :
        pages_to_pull = random.sample(links,max_pages) 
        
    for page in pages_to_pull :
        page_text = '' # initialize it to an empty string
        
        
        
        if page and "#" in page : # Skips tabs
            
            if page[0] == r"#" : 
                page = page[1:]

               
            
        try :
            r = requests.get(page)
        except :
            pass 

        if r.status_code == 200 :
            soup = BeautifulSoup(r.text, 'html.parser')
            texts = soup.findAll(text=True)
            visible_texts = filter(tag_visible, texts) 
            page_text = " ".join(t.strip() for t in visible_texts)

        #Update DB
        sql_query = ('UPDATE site_text '
                     'SET text = ? '
                     'WHERE url = ? ' )
        
       
        cur.execute(sql_query,[page_text,page])
        
        sql_query = ('UPDATE site_text '
                     'SET pulled = 1 '
                     'WHERE url = ? ' )
        
        
        cur.execute(sql_query,[page])

        sql_query = ('UPDATE site_text '
                     'SET dt = ? '
                     'WHERE url = ? ' )
        
      
        cur.execute(sql_query,[datetime.datetime.now(),page])
        
    print("Completed pull for {}.".format(repo_sites))

db.commit()     

Completed pull for http://sustainability.stryker.com/.
Completed pull for https://www.jnjmedicaldevices.com/en-US/service/reprocessing.


Some of the links were null so i deleted the tags that were not pulled

In [11]:
sql_query= ' DELETE FROM site_text WHERE pulled=0 '
cleaner = cur.execute(sql_query)

db.commit()


Another problem was I had some links that were marked as blob and they were just miilions of random characters.  To get rid of them I did it a little none programatically.  There were only three so I just set the cell as null and then deleted the rows

In [15]:
sql_query= ' DELETE FROM site_text WHERE text IS NULL '
cleaner = cur.execute(sql_query)

db.commit()


In [6]:
db = sqlite3.connect("reprocessing_competitors.db") # creating connection to DB
cur = db.cursor()

In [9]:
stryker= cur.execute('''SELECT * FROM site_text
                                    )


OperationalError: no tables specified